# Проект №8 - Машинное обучение в бизнесе

# Описание проекта


Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

##### Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

### План проекта
1. Загрузите и подготовьте данные. Поясните порядок действий.
2. Обучите и проверьте модель для каждого региона:
    1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
    2. Обучите модель и сделайте предсказания на валидационной выборке.
    3. Сохраните предсказания и правильные ответы на валидационной выборке.
    4. Напечатайте на экране средний запас сырья и RMSE модели.
    5. Проанализируйте результаты.
3. Подготовьтесь к расчёту прибыли:
    1. Сохраните в коде все ключевые значения для расчётов.
    2. Посчитайте минимальный средний объём сырья в месторождениях региона, достаточный для его разработки. Напишите выводы.
    3. Напишите функцию для расчёта прибыли по набору отобранных месторождений и предсказаний модели.
4. Посчитайте риски и прибыль для каждого региона:
    1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков.
    3. Напишите выводы: предложите регион для разработки месторождений и обоснуйте выбор.
    
---
    
# 1. Загрузка и подготовка данных

Данные геологоразведки трёх регионов находятся в файлах: geo_data_0.csv // geo_data_1.csv // geo_data_2.csv
* id — уникальный идентификатор месторождения;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в месторождении (тыс. баррелей).

In [228]:
#начальный набор библиотек
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
sns.set()

In [229]:
import warnings
warnings.filterwarnings('ignore')

from scipy import stats as st 
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, r2_score, mean_absolute_error
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, StratifiedShuffleSplit

In [230]:
import locale
locale.setlocale(locale.LC_ALL, '')

'en_US.UTF-8'

In [231]:
filename_1 = 'geo_data_0.csv'
filename_2 = 'geo_data_1.csv'
filename_3 = 'geo_data_2.csv'
geo_data_1 = pd.read_csv(filename_1)
geo_data_2 = pd.read_csv(filename_2)
geo_data_3 = pd.read_csv(filename_3)

In [232]:
print('Размеры первого сета -', geo_data_1.shape)
print('Размеры второго сета -', geo_data_2.shape)
print('Размеры третьего сета -', geo_data_3.shape, '\n')
print('\nИнформация о первом сете')
print(geo_data_1.info())
print('\nИнформация втором сете')
print(geo_data_2.info())
print('\nИнформация третьем сете')
print(geo_data_3.info())

Размеры первого сета - (100000, 5)
Размеры второго сета - (100000, 5)
Размеры третьего сета - (100000, 5) 


Информация о первом сете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Информация втором сете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Информация третьем сете
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null o

In [233]:
geo_data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04


#### Проверка на пропуски

In [234]:
if (geo_data_1.isnull().sum().sum() + geo_data_2.isnull().sum().sum() + geo_data_3.isnull().sum().sum()) ==0:
    print('Nan - не обнаружено в сетах')

Nan - не обнаружено в сетах


#### Вывод:
Nan и None значений не обнаружено.

#### Проверим на дубли

In [235]:
if (geo_data_1.duplicated().sum() + geo_data_1.duplicated().sum() + geo_data_1.duplicated().sum()) == 0:
    print('дубликатов строк в сетах нет')
else:
    print('требуется доп.анализ, т.к. обнаружены дубликаты строк в сетах')

дубликатов строк в сетах нет


In [236]:
def duble(data):
    result = (data[data.duplicated(subset='id', keep=False) == True]
     .sort_values(by='id',ascending=False))
    return result

duble(geo_data_1)

,id,f0,f1,f2,product
90815,fiKDv,0.05,0.84,6.39,137.35
16633,fiKDv,0.16,1.03,5.59,95.82
41724,bxg6G,-0.82,0.55,3.63,93.01
1364,bxg6G,0.41,0.86,-3.65,73.60
97785,bsk9y,0.38,0.01,0.16,160.64
89582,bsk9y,0.40,-0.40,10.12,163.43
92341,TtcGQ,0.11,1.02,0.91,101.32
60140,TtcGQ,0.57,-0.10,6.44,85.35
21426,Tdehs,0.83,0.30,-0.05,96.04
75715,Tdehs,0.11,0.43,3.22,60.96


In [237]:
duble(geo_data_2)

,id,f0,f1,f2,product
47591,wt4Uk,-9.09,-8.11,-0.00,3.18
82873,wt4Uk,10.26,-9.38,4.99,134.77
2721,bfPNe,-9.49,-5.46,4.01,110.99
82178,bfPNe,-6.20,-4.82,3.00,84.04
1305,LHZR0,11.17,-1.95,3.00,80.86
41906,LHZR0,-8.99,-4.29,2.01,57.09
5849,5ltQ6,-3.44,-12.30,2.00,57.09
84461,5ltQ6,18.21,2.19,3.99,107.81


In [238]:
duble(geo_data_3)

,id,f0,f1,f2,product
28039,xCHr8,1.63,0.37,-2.38,6.12
43233,xCHr8,-0.85,2.10,5.60,184.39
44378,Vcm5J,-1.23,-2.44,1.22,137.97
95090,Vcm5J,2.59,1.99,2.48,92.33
11449,VF7Jo,2.12,-0.86,5.75,181.72
49564,VF7Jo,-0.88,0.56,0.72,136.23
45404,KUPhW,0.23,-1.70,4.99,11.72
55967,KUPhW,1.21,3.18,5.54,132.83


In [239]:
print('Процент дублирующихся идентификаторов - {:.3%}'.format(\
(len(geo_data_1['id'].value_counts()[ geo_data_1['id'].value_counts() >= 2].index) +\
len(geo_data_2['id'].value_counts()[ geo_data_2['id'].value_counts() >= 2].index) +\
len(geo_data_3['id'].value_counts()[ geo_data_3['id'].value_counts() >= 2].index)) /\
                                                              (len(geo_data_1)+len(geo_data_2)+len(geo_data_3))\
                                                             )\
     )

Процент дублирующихся идентификаторов - 0.006%


In [240]:
geo_data_1['region'] = 1
geo_data_2['region'] = 2
geo_data_3['region'] = 3
data = pd.merge(pd.merge(geo_data_1,geo_data_2, how='outer'), geo_data_3, how='outer')
print('size new data', data.shape)
data.head()

size new data (300000, 6)


,id,f0,f1,f2,product,region
0,txEyH,0.71,-0.50,1.22,105.28,1
1,2acmU,1.33,-0.34,4.37,73.04,1
2,409Wp,1.02,0.15,1.42,85.27,1
3,iJLyR,-0.03,0.14,2.98,168.62,1
4,Xdl7t,1.99,0.16,4.75,154.04,1


In [241]:
print('Процент дублирующихся идентификаторов - {:.3%}'.format(\
(len(data['id'].value_counts()[data['id'].value_counts() >= 2].index)) / (len(data))))

problem = data[data['id'].isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index)]
problem.pivot_table(index=['id','region'])

Процент дублирующихся идентификаторов - 0.016%


f0    f1    f2  product
id    region                           
2tyMi 1       0.58 -0.41 -3.73    69.29
      3      -1.79 -1.36 -4.84   145.90
5ltQ6 2       7.39 -5.05  3.00    82.45
5ssQt 1      -1.01  0.27 -0.41    53.21
      3      -0.65  0.78  2.69   120.11
...            ...   ...   ...      ...
uSye4 3      -0.07  2.54 -0.05   141.35
wqgPo 1       2.01 -0.12  5.25    72.80
      3       0.05  1.42  0.09    10.69
wt4Uk 2       0.58 -8.74  2.50    68.97
xCHr8 3       0.39  1.23  1.61    95.25

[80 rows x 4 columns]

Данные записи являются аномальными, т.к. присутствуют в разных регионах. Удалим эти записи из общего датасета, чтобы не создавать аномалий при обучении.

In [242]:
data = data[~data['id'].isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index)]
data_1 = geo_data_1[(~geo_data_1['id']
                     .isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index))]
data_2 = geo_data_2[(~geo_data_2['id']
                     .isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index))]
data_3 = geo_data_3[(~geo_data_3['id']
                     .isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index))]

# 2. Обучение и проверка модели

##### Условия задачи:
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона проводится исследование 500 точек.
* Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей.
* Один баррель сырья приносит 4500 рублей прибыли.
* Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.

### Разделим выборки

In [243]:
features_1 = data_1.drop(['product', 'id'], axis=1)
target_1 = data_1['product']
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(\
                                                    features_1, target_1,\
                                                    test_size=.25, random_state=12345)

features_2 = data_2.drop(['product', 'id'], axis=1)
target_2 = data_2['product']
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(\
                                                    features_2, target_2,\
                                                    test_size=.25, random_state=12345)
features_3 = data_3.drop(['product', 'id'], axis=1)
target_3 = data_3['product']
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(\
                                                    features_3, target_3,\
                                                    test_size=.25, random_state=12345)

In [244]:
def scaling_features(x_train, x_test):
    scaler = StandardScaler()
    num_features = ['f0','f1','f2','region']
    x_train[num_features] = scaler.fit_transform(x_train[num_features])
    x_test[num_features] = scaler.transform(x_test[num_features])
    return x_train, x_test

x_train_1, x_test_1 = scaling_features(x_train_1, x_test_1)
x_train_2, x_test_2 = scaling_features(x_train_2, x_test_2 )
x_train_3, x_test_3 = scaling_features(x_train_3, x_test_3 )

In [245]:
print('Размер обучающей выборки', x_train_1.shape, '// Размер целевой выборки', y_train_1.shape)
print('Размер тренировочной выборки', x_test_1.shape, '// Размер целевой выборки', y_test_1.shape)

Размер обучающей выборки (75000, 4) // Размер целевой выборки (75000,)
Размер тренировочной выборки (25000, 4) // Размер целевой выборки (25000,)


### Блок функций

In [246]:
def func_grid_create_table(grid_cv):
    '''
    Функция возвращает датасет результата работы RandomForestRegressor и удаляет лишние признаки
    '''
    table = pd.DataFrame(grid_cv.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
    table = table.drop([
                    'mean_fit_time', 
                    'std_fit_time', 
                    'mean_score_time',
                    'std_score_time', 
                    'params', 
                    'split0_test_score', 
                    'split1_test_score', 
                    'split2_test_score', 
                    'std_test_score'],
                    axis=1)
    table['param_max_depth'] = table['param_max_depth'].astype('int64')
    table['param_n_estimators'] = table['param_n_estimators'].astype('int64')
    sns.heatmap(table.loc[:,['param_max_depth','param_n_estimators','mean_test_score']].corr())
    print('size table:', table.shape)
    return table

def rfr_simple(x_train, y_train, x_test, y_test, ind, region):
    model = RandomForestRegressor()
    model.fit(x_train, y_train)

    pred = model.predict(x_test)
    print('\n Веса:')
    print(model.feature_importances_,)
    
    result.loc[ind] = ['model_RFR_'+region,\
                       'RFR',\
                       r2_score(y_test, pred),\
                       mean_absolute_error(y_test, pred),\
                       mse(y_test, pred)**(.5),\
                       region]
    ind+=1
    return model, result, ind, pred

def LR(x_train, y_train, x_test, y_test, ind, region):
    model = LinearRegression()
    model.fit(x_train, y_train)
    pred = model.predict(x_test)
    result.loc[ind] = ['model_linear_'+ region,\
                       'LR',\
                       r2_score(y_test, pred),\
                       mean_absolute_error(y_test, pred),\
                       mse(y_test, pred)**(.5),\
                       region]
    ind+=1
    return model, result, ind, pred

## RandomForestRegressor и LinearRegression для трёх регионов

In [247]:
#Параметры для GridSearch
param_dist = {'n_estimators': [int(x) for x in np.linspace(start = 1, stop = 100, num = 2)],
               #'max_features': ['log2', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(start = 1, stop = 100, num = 2)],
               #'min_samples_split': [int(x) for x in np.linspace(start = 2, stop = 50, num = 25)],
               #'min_samples_leaf': [int(x) for x in np.linspace(start = 2, stop = 50, num = 25)],
               #'bootstrap': [True, False]
             }

#### 1. RFR для первого региона

In [248]:
result = pd.DataFrame(columns=['name', 'type', 'r2', 'mae', 'rmse', 'region'])
ind = 0

#### Воспользуемся моделью случайного леса без гиперпараметров

In [249]:
%%time

model_rfr_reg1, result, ind, pred_rfr_reg1 = rfr_simple(x_train_1, y_train_1,\
                                                         x_test_1, y_test_1, ind, 'r1')
result


 Веса:
[0.25543459 0.27231177 0.47225364 0.        ]
CPU times: user 4 s, sys: 20 ms, total: 4.02 s
Wall time: 4.02 s


,name,type,r2,mae,rmse,region
0,model_RFR_r1,RFR,0.17,32.73,40.37,r1


#### Воспользуемся линейной регрессией

In [250]:
model_LR_reg1, result, ind, pred_LR_reg1 = LR(x_train_1, y_train_1,\
                                               x_test_1, y_test_1, ind, 'r1')
result

,name,type,r2,mae,rmse,region
0,model_RFR_r1,RFR,0.17,32.73,40.37,r1
1,model_linear_r1,LR,0.28,30.92,37.58,r1


#### 2. RFR для второго региона

#### Воспользуемся моделью случайного леса без гиперпараметров

In [251]:
model_rfr_reg2, result, ind, pred_rfr_reg2 = rfr_simple(x_train_2, y_train_2, x_test_2,\
                                                        y_test_2, ind, 'r2')
result


 Веса:
[1.03917840e-03 8.46768293e-05 9.98876145e-01 0.00000000e+00]


,name,type,r2,mae,rmse,region
0,model_RFR_r1,RFR,0.17,32.73,40.37,r1
1,model_linear_r1,LR,0.28,30.92,37.58,r1
2,model_RFR_r2,RFR,1.00,0.33,0.78,r2


#### Воспользуемся линейной регрессией

In [252]:
model_LR_reg2, result, ind, pred_LR_reg2  = LR(x_train_2, y_train_2, x_test_2, y_test_2, ind, 'r2')
result

,name,type,r2,mae,rmse,region
0,model_RFR_r1,RFR,0.17,32.73,40.37,r1
1,model_linear_r1,LR,0.28,30.92,37.58,r1
2,model_RFR_r2,RFR,1.00,0.33,0.78,r2
3,model_linear_r2,LR,1.00,0.72,0.89,r2


#### 3. RFR для третьего региона

#### Воспользуемся моделью случайного леса без гиперпараметров

In [253]:
%%time
result_rfr_reg3 = rfr_simple(x_train_3, y_train_3, x_test_3, y_test_3, ind, 'r3')
ind = result_rfr_reg3[2]
result = result_rfr_reg3[1]
result


 Веса:
[0.2818153 0.2856099 0.4325748 0.       ]
CPU times: user 4.27 s, sys: 4 ms, total: 4.28 s
Wall time: 4.29 s


,name,type,r2,mae,rmse,region
0,model_RFR_r1,RFR,0.17,32.73,40.37,r1
1,model_linear_r1,LR,0.28,30.92,37.58,r1
2,model_RFR_r2,RFR,1.00,0.33,0.78,r2
3,model_linear_r2,LR,1.00,0.72,0.89,r2
4,model_RFR_r3,RFR,0.17,33.37,40.93,r3


#### Воспользуемся линейной регрессией

In [254]:
model_LR_reg3, result, ind, pred_LR_reg3  = LR(x_train_3, y_train_3,\
                                               x_test_3, y_test_3, ind, 'r3')
result.sort_values(by='r2', ascending=False)

,name,type,r2,mae,rmse,region
2,model_RFR_r2,RFR,1.00,0.33,0.78,r2
3,model_linear_r2,LR,1.00,0.72,0.89,r2
1,model_linear_r1,LR,0.28,30.92,37.58,r1
5,model_linear_r3,LR,0.21,32.79,40.03,r3
4,model_RFR_r3,RFR,0.17,33.37,40.93,r3
0,model_RFR_r1,RFR,0.17,32.73,40.37,r1


#### Сделаем подсчёт среднего запаса сырья в каждом регионе

In [255]:
 
def func(row):
    if row.region == 'r1':
        return target_1.mean()*1000
    elif row.region == 'r2':
        return target_2.mean()*1000
    else:
        return target_3.mean()*1000
        
    
result['stock_of_raw_materials'] = result.apply(func, axis=1)
result

,name,type,r2,mae,rmse,region,stock_of_raw_materials
0,model_RFR_r1,RFR,0.17,32.73,40.37,r1,"92,500.00"
1,model_linear_r1,LR,0.28,30.92,37.58,r1,"92,500.00"
2,model_RFR_r2,RFR,1.00,0.33,0.78,r2,"68,825.00"
3,model_linear_r2,LR,1.00,0.72,0.89,r2,"68,825.00"
4,model_RFR_r3,RFR,0.17,33.37,40.93,r3,"95,000.00"
5,model_linear_r3,LR,0.21,32.79,40.03,r3,"95,000.00"


In [256]:
#Самый большой запас сырья
result[\
       (result['stock_of_raw_materials'] == result['stock_of_raw_materials'].max()) & \
       (result['type'] == 'LR')]

,name,type,r2,mae,rmse,region,stock_of_raw_materials
5,model_linear_r3,LR,0.21,32.79,40.03,r3,"95,000.00"


In [257]:
result[(result['type'] == 'LR')].sort_values(by='stock_of_raw_materials', ascending=False)

,name,type,r2,mae,rmse,region,stock_of_raw_materials
5,model_linear_r3,LR,0.21,32.79,40.03,r3,"95,000.00"
1,model_linear_r1,LR,0.28,30.92,37.58,r1,"92,500.00"
3,model_linear_r2,LR,1.00,0.72,0.89,r2,"68,825.00"


Второй регион самый невыгодный с точки зрения освоения, т.к. объём запасов меньше на порядок 20'000 в сравнеии с первым и третьим

#### Оценка качества

In [258]:
result.sort_values(by='r2', ascending=False)

,name,type,r2,mae,rmse,region,stock_of_raw_materials
2,model_RFR_r2,RFR,1.00,0.33,0.78,r2,"68,825.00"
3,model_linear_r2,LR,1.00,0.72,0.89,r2,"68,825.00"
1,model_linear_r1,LR,0.28,30.92,37.58,r1,"92,500.00"
5,model_linear_r3,LR,0.21,32.79,40.03,r3,"95,000.00"
4,model_RFR_r3,RFR,0.17,33.37,40.93,r3,"95,000.00"
0,model_RFR_r1,RFR,0.17,32.73,40.37,r1,"92,500.00"


Во втором регионе rfr работает идеально и это повод дополнительно их исследовать. В любом случае линейная регрессия работает более качественно по метрикам.

In [259]:
model_result = result[(result.type=='LR')].sort_values(by='r2', ascending=False)
model_result

,name,type,r2,mae,rmse,region,stock_of_raw_materials
3,model_linear_r2,LR,1.00,0.72,0.89,r2,"68,825.00"
1,model_linear_r1,LR,0.28,30.92,37.58,r1,"92,500.00"
5,model_linear_r3,LR,0.21,32.79,40.03,r3,"95,000.00"


## Вывод
Модели обучены, метрики получены. По качеству модель второго и первого региона работают наравне на основе метрики r2. Самый большой запас сырья у третьего, затем у второго.

<font color='green'>Да, верно. </font>


# 3. Подготовка к расчёту прибыли

#### Условия задачи:


* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона проводится исследование 500 точек.
* Бюджет на разработку месторождений — 10 млрд рублей, стоимость бурения одной скважины — 50 млн рублей.
* Один баррель сырья приносит 4500 рублей прибыли.
* Не рассматривать регионы, в которых риск убытков выше 2.5%. Из оставшихся выбирается регион с наибольшей средней прибылью.

In [260]:
POINT_PER_REGION = 500
BUDGET = 10000000000
COST_PER_WELL = 50000000
COST_ONE_BAREL = 4500
RISK = 0.025

In [261]:
#well
BUDGET/COST_PER_WELL

200.0

Найдём минимальный объём барелей для региона

In [262]:
def roi(revenue, investment):
    return (revenue-investment)/investment
    
roi_coef = 1    
for amount in range(1000, 1000000000000, 1):
    if roi(COST_ONE_BAREL*amount, BUDGET)>0:
        print('Необходимый запас барелей, чтобы произошла окупаемость 200 скважин =',amount)
        print('Необходимый запас барелей на одну скважину =', round(amount/200,0))
        print('ROI = {:.2f}'.format(roi(COST_ONE_BAREL*amount, BUDGET)))
        break
    

   


Необходимый запас барелей, чтобы произошла окупаемость 200 скважин = 2222223
Необходимый запас барелей на одну скважину = 11111.0
ROI = 0.00


In [263]:
pred_LR_reg1_s = pd.Series(pred_LR_reg1, index=range(25000))
pred_LR_reg2_s = pd.Series(pred_LR_reg2, index=range(25000))
pred_LR_reg3_s = pd.Series(pred_LR_reg3, index=range(25000))

y_test_1_s = pd.Series(y_test_1).reset_index(drop=True)
y_test_2_s = pd.Series(y_test_2).reset_index(drop=True)
y_test_3_s = pd.Series(y_test_3).reset_index(drop=True)

In [264]:
sample_pred = {\
               'predictions_linear_r1':pred_LR_reg1_s,\
               'predictions_linear_r2':pred_LR_reg2_s,\
               'predictions_linear_r3':pred_LR_reg3_s}
sample_y = {\
            'y_test_1':y_test_1_s,\
            'y_test_2':y_test_2_s,\
            'y_test_3':y_test_3_s}

zips = zip(sample_y.values(), sample_pred.values())
zips

In [265]:
print(pred_LR_reg1_s.index == y_test_1_s.index)
print(pred_LR_reg2_s.index == y_test_2_s.index)
print(pred_LR_reg3_s.index == y_test_3_s.index)

[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]


In [266]:
pred_LR_reg1_s

0        95.89
1        77.57
2        77.89
3        90.18
4        70.51
         ...  
24995   103.04
24996    85.40
24997    61.51
24998   118.18
24999   118.17
Length: 25000, dtype: float64

In [267]:
y_test_1_s

0        10.04
1       114.55
2       132.60
3       169.07
4       122.33
         ...  
24995   170.12
24996    93.63
24997   127.35
24998    99.78
24999   177.82
Name: product, Length: 25000, dtype: float64

In [268]:
#Дополнительно умножил на тысячу, т.к измерения в тысячах
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return COST_ONE_BAREL * selected.sum()*1000 

def gross_profit(revenue, cost):
    return revenue - cost

def mean_revenue_and_std(target, probabilities, max_point_well=500, well=200, quantile=0.05):

    state = np.random.RandomState(12345)

    #храним выручку
    values = []
    for i in range(1000):
        #поиск средней выручки
        target_subsample = target.sample(n=max_point_well, replace=True, random_state=state)
        probs_subsample = probabilities[target_subsample.index]
        #values.append(revenue(target_subsample, probs_subsample, well))
        values.append(revenue(target, probs_subsample, well))
    
    values = pd.Series(values)
    
    loss = len(values[values<0])/len(values)
    print('loss',loss)
    
    lower = values.quantile(0.05)
    
    #Выручка
    mean = values.mean()
    print("Средняя выручка:", locale.format('%d', mean, grouping=True))
    
    #Коэффициент возврата инвестиций
    roi_coef = abs(roi(BUDGET, mean))
    print('roi_coef', roi_coef)
    
    #нижняя граница доверительного интервала прибыли - 95%
    lower_private = values.quantile(.025) - BUDGET #ИСПРАВИЛ
    print('lower_private',locale.format('%d', lower_private, grouping=True))
    
    #верхняя граница доверительного интервала прибыли - 95%
    upper_private = values.quantile(.975) - BUDGET   #ИСПРАВИЛ
    print('upper_private',locale.format('%d', upper_private, grouping=True))
    
    gross_prof = gross_profit(mean, BUDGET) 
    print('gross_prof',locale.format('%d', gross_prof, grouping=True))

    #print(str(quantile*100)+"%-квантиль:", locale.format('%d', lower, grouping=True))
    return values, mean, lower_private, upper_private, gross_prof, roi_coef

# 4. Расчёт прибыли и рисков 

In [269]:
reg = 0
revenue_df = pd.DataFrame(columns=['net_conf_down','gross_profit', 'net_conf_up', 'revenue', 'ROI'])

for y, prob_y in zips:
    reg+=1
    print('\nРегион '+str(reg)+':')
    values, mean, d, up , gross, r = mean_revenue_and_std(y, prob_y);
    revenue_df.loc[reg] = [d, gross, up, mean, r]
        
pd.options.display.float_format = '{:,.2f}'.format    
revenue_df


Регион 1:
loss 0.0
Средняя выручка: 103,961,649,848
roi_coef 0.9038106838952777
lower_private 88,887,844,541
upper_private 99,097,669,415
gross_prof 93,961,649,848

Регион 2:
loss 0.0
Средняя выручка: 104,560,451,057
roi_coef 0.9043615449357069
lower_private 90,338,205,093
upper_private 98,522,894,538
gross_prof 94,560,451,057

Регион 3:
loss 0.0
Средняя выручка: 104,044,038,665
roi_coef 0.9038868528341907
lower_private 88,366,495,866
upper_private 99,503,595,749
gross_prof 94,044,038,665


,net_conf_down,gross_profit,net_conf_up,revenue,ROI
1,"88,887,844,541.10","93,961,649,848.02","99,097,669,415.53","103,961,649,848.02",0.90
2,"90,338,205,093.99","94,560,451,057.87","98,522,894,538.66","104,560,451,057.87",0.90
3,"88,366,495,866.04","94,044,038,665.68","99,503,595,749.24","104,044,038,665.68",0.90


In [270]:
revenue_df

,net_conf_down,gross_profit,net_conf_up,revenue,ROI
1,"88,887,844,541.10","93,961,649,848.02","99,097,669,415.53","103,961,649,848.02",0.90
2,"90,338,205,093.99","94,560,451,057.87","98,522,894,538.66","104,560,451,057.87",0.90
3,"88,366,495,866.04","94,044,038,665.68","99,503,595,749.24","104,044,038,665.68",0.90


In [271]:
model_result

,name,type,r2,mae,rmse,region,stock_of_raw_materials
3,model_linear_r2,LR,1.00,0.72,0.89,r2,"68,825.00"
1,model_linear_r1,LR,0.28,30.92,37.58,r1,"92,500.00"
5,model_linear_r3,LR,0.21,32.79,40.03,r3,"95,000.00"


Выручка с вероятностью 95% во всех регионах ожидается и находится для всех регионов в доверительном интервале с нижней границей - __net_conf_down__, и верхней - __net_conf_up__. Риск убытка не прогнозируется(параметр __loss__).
Выбор падает на третий регион, т.к. он обладает самым большим запасом и ROI.